## Keypoint Detection
---
There are 5 images given in the ***img*** folder. Your task is to detect ***200*** keypoints for each of them using detectors.

You are free to use any languages (C, Matlab, Python), any keypoint detection methods (such as SIFT, ORB, ...)

Let's take a look at these images first!

---

In [1]:
# load packages
import cv2
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from numpy import array
%matplotlib inline
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
img_dir = "image_retrieval/images"
if os.path.exists(img_dir):
    if os.listdir(img_dir) is []:
        print("No images!")
        exit(0)
    num_img = len(os.listdir(img_dir))
    path_list=os.listdir(img_dir)
    path_list.sort()
    
    keypoints_list = []
    for k in range(1, 141):
        img = str(k) + ".JPG"
        image_dir = os.path.join(img_dir, img)
        image = cv2.imread(image_dir)
        gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)        
        
        sift = cv2.xfeatures2d.SIFT_create(30)
        kps, des = sift.detectAndCompute(gray,None)
        
        keypoints = [kp.pt for kp in kps]
        plt.show()
        keypoints_list.append(keypoints)
        
    tensor = keypoints_list
    output_dir = "wangxint_keypoints_140.pt"
    torch.save(tensor, output_dir)

else:
    print("image folder not exists!")
    exit(0)

### The Keypoints:
The output of this part is a list of keypoints that represented by the x and y cooridnates.

For example, the 10 points of image ***NotreDame5.jpg*** are:

In [3]:
#keypoints = list([(1896.0, 1815.0), (1614.0, 1164.0), (1471.0, 964.0), (2099.0, 1032.0), (2080.0, 980.0), (1462.0, 925.0), (1707.0, 1057.0), (1840.0, 1138.0), (1309.0, 1105.0), (1959.0, 1778.0)])
# keypoints = list([(20, 400)])

Then we can obtain the patches with these keypoints:


In [4]:
def getPatches(kps, img, size=32, num=30):
    res = torch.zeros(num, 1, size, size)
    if type(img) is np.ndarray:
        img = torch.from_numpy(img)
    h, w = img.shape      # note: for image, the x direction is the verticle, y-direction is the horizontal...
    for i in range(num):
        cx, cy = kps[i]
        cx, cy = int(cx), int(cy)
        dd = int(size/2)
        xmin, xmax = max(0, cx - dd), min(w, cx + dd ) - 1
        ymin, ymax = max(0, cy - dd), min(h, cy + dd ) - 1 
        
        xmin_res, xmax_res = dd - min(dd,cx), dd + min(dd, w - cx)-1
        ymin_res, ymax_res = dd - min(dd,cy), dd + min(dd, h - cy)-1

        res[i, 0, ymin_res: ymax_res, xmin_res: xmax_res] = img[ymin: ymax, xmin: xmax]
    return res

### Let's plot these patches

In [5]:
img_dir = "image_retrieval/images"
all_patches = []
num_img = len(os.listdir(img_dir))
i = 0
for k in range(1, 141):
    img = str(k) + ".JPG"
    image_dir = os.path.join(img_dir, img)
    image = cv2.imread(image_dir)
    gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)    
    patches = getPatches(keypoints_list[i], gray,size=32, num=30)
    i += 1
    
    all_patches.append(patches)
#    torch.save(patches, output_dir)
    
print(type(all_patches))
tensor_patches=torch.stack((all_patches))
print(type(tensor_patches))


<class 'list'>
<class 'torch.Tensor'>


### Save the patches with PyTorch
For each image, you can output the patches within one tensor. In above examples, tensor ***patches*** is the one that you should store in a list. And then save the list as a "patches.pt" file:

In [6]:
output_dir = "wangxint_patches_140.pt"
torch.save(tensor_patches, output_dir)


### Test with your saved patches

In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
test_patches = torch.load("wangxint_patches_140.pt")
print(type(test_patches))
print(test_patches.shape)

<class 'torch.Tensor'>
torch.Size([140, 30, 1, 32, 32])


In [8]:
from descriptor_CNN3 import DesNet as Model

model = Model()

model.cuda()
model.eval()
models = [model]
path = "./HM2/checkpoint.pth"

weights = torch.load(path)
model.load_state_dict(weights['state_dict'])
patches = test_patches.view(-1, 1, 32, 32).cuda()
print(patches.shape)
print(type(patches))
with torch.no_grad():
    output = model(patches)
    #output.shape = torch.Size([1000, 128])
out = output.view(140, 30, 128).cpu().data

output_dir = "wangxint_descriptions_140.pt"
torch.save(out, output_dir)

torch.Size([4200, 1, 32, 32])
<class 'torch.Tensor'>
DesNet(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1,

tensor([[[-0.0076, -0.0201,  0.0595,  ..., -0.0723, -0.0697, -0.1254],
         [ 0.0505, -0.0634,  0.1656,  ...,  0.2073, -0.0765, -0.1073],
         [-0.0076, -0.0201,  0.0595,  ..., -0.0723, -0.0697, -0.1254],
         ...,
         [ 0.0629,  0.0918, -0.0257,  ..., -0.0250, -0.1794,  0.0120],
         [ 0.0629,  0.0918, -0.0257,  ..., -0.0250, -0.1794,  0.0120],
         [ 0.0006,  0.0785, -0.0118,  ...,  0.0232, -0.0420, -0.0018]],

        [[ 0.0545,  0.0141,  0.0821,  ...,  0.1271, -0.0307, -0.1165],
         [-0.0028,  0.0102,  0.0500,  ..., -0.0536, -0.1407, -0.0956],
         [-0.0330,  0.0455, -0.0282,  ..., -0.0664, -0.1017, -0.1522],
         ...,
         [ 0.0239,  0.0531,  0.1338,  ...,  0.0507, -0.0435, -0.1575],
         [ 0.0060,  0.0844,  0.0048,  ..., -0.0286, -0.0823,  0.0117],
         [ 0.0060,  0.0844,  0.0048,  ..., -0.0286, -0.0823,  0.0117]],

        [[ 0.0555, -0.0674,  0.2441,  ...,  0.0416, -0.0557, -0.0788],
         [ 0.0243, -0.1494,  0.0992,  ...,  0

# 

torch.Size([35, 30, 1, 32, 32])
tensor([[[[[113.,  83., 133.,  ..., 165., 168.,   0.],
           [ 99., 161., 177.,  ..., 170., 169.,   0.],
           [116.,  78., 100.,  ..., 166., 169.,   0.],
           ...,
           [174., 166., 187.,  ..., 125., 168.,   0.],
           [176., 160., 172.,  ..., 132., 165.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.]]],


         [[[174., 138., 106.,  ..., 149., 183.,   0.],
           [118.,  87., 105.,  ..., 128., 182.,   0.],
           [111., 102.,  91.,  ..., 108., 180.,   0.],
           ...,
           [189., 192., 193.,  ..., 138., 142.,   0.],
           [193., 192., 189.,  ..., 146., 143.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.]]],


         [[[183., 184., 184.,  ..., 147., 143.,   0.],
           [186., 187., 181.,  ..., 143., 144.,   0.],
           [182., 182., 181.,  ..., 146., 150.,   0.],
           ...,
           [ 97., 168., 172.,  ..., 156., 177.,   0.],
           [112.,  85.,  95.,  .